<a href="https://colab.research.google.com/github/Mohit2004Gothwal/Restaurant-Info-Chatbot-for-Lucknow/blob/main/zomato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MOHIT KUMAR LIT2022032  ZOMATO INTERNSHIP ASSIGNMENT

#Web Scraping (from public restaurant pages)

There are 6 Zomato Connected Restaurant

1. The Terrace
2. The Big Grill
3. Barbeque Nation
4. Cafe HONS - House of No Sugar
5. The Penthouse Cafe
6. HONS - All Day Dining

In [ ]:
import json


In [ ]:
# Initial 3 Restaurants
restaurants = [
    {
        "name": "The Terrace",
        "location": "Hazratganj, Lucknow",
        "contact": "+91 8400020242",
        "hours": "12noon – 11pm",
        "features": ["Vegetarian Options", "Live Music", "Rooftop Seating"],
        "menu": [
            {"item": "Paneer Lababdar", "description": "Cottage cheese in creamy tomato gravy", "price": "₹320"},
            {"item": "Dal Makhani", "description": "Black lentils cooked overnight", "price": "₹260"},
            {"item": "Butter Naan", "description": "Soft tandoori bread", "price": "₹60"},
        ]
    },
    {
        "name": "The Big Grill",
        "location": "Gomti Nagar, Lucknow",
        "contact": "+91 7408428333",
        "hours": "12noon – 11pm",
        "features": ["Buffet", "Non-Veg", "Spicy Grill Specials"],
        "menu": [
            {"item": "Chicken Tandoori", "description": "Spiced grilled chicken", "price": "₹350"},
            {"item": "Mutton Seekh Kebab", "description": "Minced lamb skewers", "price": "₹400"},
            {"item": "Veg Biryani", "description": "Fragrant rice with vegetables", "price": "₹290"},
        ]
    },
    {
        "name": "Barbeque Nation",
        "location": "Aliganj, Lucknow",
        "contact": "+91 9876543210",
        "hours": "12noon – 3:30pm, 6:30pm – 11pm",
        "features": ["Live Grill", "Buffet", "Veg & Non-Veg", "Allergen Info"],
        "menu": [
            {"item": "Pineapple Grill", "description": "Grilled pineapple with spices", "price": "₹280"},
            {"item": "Fish Tikka", "description": "Boneless fish with Indian spices", "price": "₹390"},
            {"item": "Brownie with Ice Cream", "description": "Hot brownie & vanilla ice cream", "price": "₹190"},
        ]
    }
]


In [ ]:
# Add 3 more restaurants based on your new list
more_restaurants = [
    {
        "name": "Cafe HONS - House of No Sugar",
        "location": "Gomti Nagar, Lucknow",
        "contact": "+91 9511113486",
        "hours": "11am – 11pm",
        "features": ["Sugar-Free Options", "Healthy Bites", "Vegan Friendly"],
        "menu": [
            {"item": "Keto Brownie", "description": "Low-carb chocolate dessert", "price": "₹180"},
            {"item": "Avocado Toast", "description": "Whole grain bread with fresh avocado", "price": "₹220"},
            {"item": "Herbal Tea", "description": "Natural detox infusion", "price": "₹100"},
        ]
    },
    {
        "name": "The Penthouse Cafe",
        "location": "Aliganj, Lucknow",
        "contact": "+91 7754067756",
        "hours": "11am – 10:30pm",
        "features": ["Rooftop Seating", "Instagrammable", "Live Music"],
        "menu": [
            {"item": "Veggie Burger", "description": "Loaded burger with fresh veggies", "price": "₹260"},
            {"item": "Fries Platter", "description": "3 styles of fries", "price": "₹180"},
            {"item": "Oreo Shake", "description": "Creamy shake with Oreo bits", "price": "₹150"},
        ]
    },
    {
        "name": "HONS - All Day Dining",
        "location": "Hazratganj, Lucknow",
        "contact": "+91 9511113486",
        "hours": "10am – 11pm",
        "features": ["All-Day Menu", "Sugar-Free Desserts", "Family Friendly"],
        "menu": [
            {"item": "Grilled Chicken Salad", "description": "Lean protein with fresh greens", "price": "₹280"},
            {"item": "Zucchini Pasta", "description": "Gluten-free spiral pasta", "price": "₹300"},
            {"item": "Sugar-Free Cheesecake", "description": "Baked dessert with stevia", "price": "₹240"},
        ]
    }
]

# Append to the original restaurant list
restaurants.extend(more_restaurants)

# Save updated list to JSON
with open("restaurants.json", "w", encoding='utf-8') as f:
    json.dump(restaurants, f, indent=4, ensure_ascii=False)

print("✅ All 6 restaurants saved to 'restaurants.json'")


✅ All 6 restaurants saved to 'restaurants.json'


In [ ]:
# Load and view saved JSON
with open("restaurants.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Print sample
for res in data:
    print(res["name"], "=>", len(res["menu"]), "menu items")


The Terrace => 3 menu items
The Big Grill => 3 menu items
Barbeque Nation => 3 menu items
Cafe HONS - House of No Sugar => 3 menu items
The Penthouse Cafe => 3 menu items
HONS - All Day Dining => 3 menu items


# Knowledge Based Creation

In [ ]:
!pip install -q faiss-cpu sentence-transformers


In [ ]:
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load restaurants.json
with open("restaurants.json", "r", encoding="utf-8") as f:
    restaurants = json.load(f)

# Flatten data into text chunks for RAG
documents = []
metadata = []

for res in restaurants:
    base = f"{res['name']} ({res['location']})\nHours: {res['hours']}\nContact: {res['contact']}\nFeatures: {', '.join(res['features'])}\n"

    for item in res['menu']:
        text = f"{base}Menu Item: {item['item']}\nDescription: {item['description']}\nPrice: {item['price']}"
        documents.append(text)
        metadata.append({"restaurant": res["name"], "menu_item": item["item"]})


In [ ]:
# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Embed all documents
embeddings = model.encode(documents, show_progress_bar=True)

# Convert to float32 numpy array (required by FAISS)
embedding_matrix = np.array(embeddings).astype("float32")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Initialize FAISS index
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings
index.add(embedding_matrix)

print(f"✅ FAISS index created with {index.ntotal} documents.")


✅ FAISS index created with 18 documents.


# Building the RAG Chatbot

In [ ]:
!pip install -q transformers


In [ ]:
from transformers import pipeline

# Load a small text generation model
generator = pipeline('text2text-generation', model='google/flan-t5-small')


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def retrieve_context(query, top_k=3):
    """Retrieve top_k similar documents for the query"""
    query_embedding = model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [documents[i] for i in indices[0]]
    return "\n\n".join(retrieved_docs)

def generate_answer(query):
    """Retrieve context and generate answer"""
    context = retrieve_context(query)
    prompt = f"Answer the question based on the following restaurant information:\n\n{context}\n\nQuestion: {query}"
    response = generator(prompt, max_length=300, do_sample=True)[0]['generated_text']
    return response


In [ ]:
while True:
    user_query = input("\nAsk about restaurants 🍽️ (or type 'exit' to quit): ")
    if user_query.lower() == "exit":
        break
    answer = generate_answer(user_query)
    print("\n🤖:", answer)



Ask about restaurants 🍽️ (or type 'exit' to quit): which one having high rating? 

🤖: The Big Grill

Ask about restaurants 🍽️ (or type 'exit' to quit): which of the following having only veg food

🤖: sushi

Ask about restaurants 🍽️ (or type 'exit' to quit): which restaurant selling only veg food

🤖: The Bakery

Ask about restaurants 🍽️ (or type 'exit' to quit): which one is good big grill or terrace

🤖: The big grill (Gomti Nagar, Lucknow) Hours: 12noon – 11pm Contact: +91 7408428333 Features: Buffet, Non-Veg, Spicy Grill Specials Menu Item: Chicken Tandoori

Ask about restaurants 🍽️ (or type 'exit' to quit): exit


# streamlit UI

In [ ]:
!pip install streamlit


In [ ]:
import streamlit as st

# Temporary dummy answer function
def generate_answer(query):
    return "Sorry, I'm still learning! You asked: " + query

# Streamlit UI
st.title("Restaurant Query Chatbot 🍽️")

st.write(
    """
    Ask about restaurant menus, prices, dietary options, and more!
    Type 'exit' to quit.
    """
)

# User input
user_query = st.text_input("Ask your question about restaurants:")

if user_query:
    answer = generate_answer(user_query)
    st.write("🤖 Answer:", answer)


2025-04-26 08:55:55.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 08:55:55.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 08:55:55.128 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 08:55:55.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 08:55:55.130 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 08:55:55.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 08:55:55.134 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 08:55:55.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!streamlit run app.py


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [ ]:
# Streamlit app code
app_code = """
import streamlit as st
from transformers import pipeline
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# Load restaurant data
with open("restaurants.json", "r", encoding="utf-8") as f:
    restaurants = json.load(f)

# Initialize model for embedding
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index
dimension = 384  # Dimension of the embedding vectors (MiniLM-L6)
index = faiss.IndexFlatL2(dimension)

# Flatten data into text chunks for RAG
documents = []
metadata = []
for res in restaurants:
    base = f"{res['name']} ({res['location']})\\nHours: {res['hours']}\\nContact: {res['contact']}\\nFeatures: {', '.join(res['features'])}\\n"
    for item in res['menu']:
        text = f"{base}Menu Item: {item['item']}\\nDescription: {item['description']}\\nPrice: {item['price']}"
        documents.append(text)
        metadata.append({"restaurant": res["name"], "menu_item": item["item"]})

# Embed all documents
embeddings = model.encode(documents, show_progress_bar=True)
embedding_matrix = np.array(embeddings).astype("float32")
index.add(embedding_matrix)

# Load a small text generation model
generator = pipeline('text2text-generation', model='google/flan-t5-small')

def retrieve_context(query, top_k=3):
    query_embedding = model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [documents[i] for i in indices[0]]
    return "\\n\\n".join(retrieved_docs)

def generate_answer(query):
    context = retrieve_context(query)
    prompt = f"Answer the question based on the following restaurant information:\\n\\n{context}\\n\\nQuestion: {query}"
    response = generator(prompt, max_length=300, do_sample=True)[0]['generated_text']
    return response

# Streamlit UI
st.title("Restaurant Query Chatbot 🍽️")
st.write("Ask about restaurant menus, prices, dietary options, and more! Type 'exit' to quit.")

# User input
user_query = st.text_input("Ask your question about restaurants:")

if user_query:
    answer = generate_answer(user_query)
    st.write("🤖 Answer: ", answer)
"""

# Write the code to app.py
with open("/content/app.py", "w") as file:
    file.write(app_code)

print("✅ app.py created successfully!")


✅ app.py created successfully!


In [ ]:
!pip install pyngrok


In [ ]:
!ngrok config add-authtoken 2wG9qx8ztvPiowhYZYzfROuUQWA_3ZB59vQGW3DkLfd445WMi


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Open tunnel
public_url = ngrok.connect(8501)
print("🔗 Open this HTTPS URL:", public_url)


🔗 Open this HTTPS URL: NgrokTunnel: "https://c7ab-34-148-98-126.ngrok-free.app" -> "http://localhost:8501"
